# Asymmetry Case Break Downs

In [21]:
from py2neo import Node, Relationship, Graph
from numpy.random import rand
import numpy as num
from bokeh.plotting import output_notebook,figure, show
from bokeh.layouts import row,column,gridplot
from bokeh.models import Label
import numpy as np
from curve import *
from bokeh.models import Span,Legend
output_notebook()

Loading BokehJS ...

In [22]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [23]:
import pickle as pk
A = pk.load(open('local.pk','rb'))


import random

def gen_hex_colour_code():
   return '#'+''.join([random.choice('0123456789ABCDEF') for x in range(6)])

C = {}

In [24]:
l = len(A)
A_dic = {}
for i in range(l):
    tmp = A[i]
    n1 = tmp['n1']
    n2 = tmp['n2']
    A_dic[n1+n2] = i

In [25]:
def roi_rot(w):
    p = w.split('L')
    return 'L'+p[-1]+'L'+p[1]
def logit(t):
    tmp = max(1,t)
    return min(0.0,num.log(tmp/5000.0))

In [26]:
tmp = ['null','0distance','goodLocal','badLocal']
cases = {}
for w in tmp:
    for v in tmp:
        z = sorted([w,v])
        cases[z[0]+'-'+z[1]] = {}
cases

{'0distance-0distance': {},
 '0distance-badLocal': {},
 '0distance-goodLocal': {},
 '0distance-null': {},
 'badLocal-badLocal': {},
 'badLocal-goodLocal': {},
 'badLocal-null': {},
 'goodLocal-goodLocal': {},
 'goodLocal-null': {},
 'null-null': {}}

In [7]:
RR = roi_regressors()
AR = all_regressors(RR,A)

/Users/kshadi3/Desktop/MANIA2/sample/curve.py:271: RuntimeWarning: invalid value encountered in double_scalars
  ce = num.sum([xx[1] for xx in A[i]['envelope']])/len(A[i]['envelope'])
/Users/kshadi3/Desktop/MANIA2/sample/curve.py:272: RuntimeWarning: invalid value encountered in double_scalars
  cex = num.sum([xx[0] for xx in A[i]['envelope']])/len(A[i]['envelope'])
/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kshadi3/Desktop/MANIA2/sample/curve.py:319: RuntimeWarning: invalid value encountered in double_scalars
  fx = (sl[0]*r[0]+(100-r[0])*tmp)/100


In [27]:
noise_floor = -4.605170185988091
def correction(AR,P):
    S = {}
    S['orig'] = logit(0)
    S['corrected'] = logit(0)
    S['n1'] = P['n1']
    S['n2'] = P['n2']
    # 0-dis flag
    dis0=False
    tmpa = num.array(P['a'])
    tmpb = num.array(P['b'])
    c = tmpa<1
    if (any(tmpb[c]>noise_floor)):
        dis0=True
        dis0val = logit(max(tmpb[c]))
    # is it null
    tmp1 = logit(P['weight'])
    if (tmp1 < noise_floor):
        S['kind'] = 'null'
        S['orig'] = logit(P['weight'])
        S['corrected'] = logit(P['weight'])
    elif (dis0):
        S['kind'] = '0distance'
        S['orig'] = dis0val
        S['corrected'] = dis0val
    elif (P['isSuccess']):
        # good local
        S['kind'] = 'goodLocal'
        S['orig'] = logit(P['weight'])
        env = P['envelope'][0]
        mod = AR[P['n1']+P['n2']]
        slope = mod[0]
        corrected = env[1] - slope*env[0]
        S['corrected'] = min(0.0,corrected)
    else:
        # bad local
        S['kind'] = 'badLocal'
        S['orig'] = logit(P['weight'])
        try:
            env = P['envelope'][0]
        except:
            env = (logit(P['weight']),P['length'])
        mod = AR[P['n1']+P['n2']]
        slope = mod[0]
        corrected = env[1] - slope*env[0]
        S['corrected'] = min(0.0,corrected)
            
    return S
    # is it 0-distance
    # is it good-local
    # is it bad-local
D = []
D_dic = {}
q = 0
for i in range(1,181):
    for j in range(1,181):
        if (i==j):continue
        n1 = 'L'+str(i)
        n2 = 'L'+str(j)
        ind = A_dic[n1+n2]
        S = correction(AR,A[ind])
        S['ind'] = ind
        D.append(S)
        D_dic[n1+n2] = q
        q = q + 1

In [28]:
nulls = len(list(filter(lambda x: x['kind']=='null',D)))
dis0 = len(list(filter(lambda x: x['kind']=='0distance',D)))
good = len(list(filter(lambda x: x['kind']=='goodLocal',D)))
bad = len(list(filter(lambda x: x['kind']=='badLocal',D)))

# All possible cases (percentage of connections)

In [29]:
print('Null Cases: '+str(nulls*100.0/32220))
print('0-distance Cases: '+str(dis0*100.0/32220))
print('Good Local: '+str(good*100.0/32220))
print('Bad Local: '+str(bad*100.0/32220))

Null Cases: 81.9522036002483
0-distance Cases: 3.258845437616387
Good Local: 11.1731843575419
Bad Local: 3.61576660459342


In [30]:
q

32220

In [44]:
cases = {}
for w in D:
    k1 = w['kind']
    n1 = w['n1']
    n2 = w['n2']
    ind2 = D_dic[n2+n1]
    w2 = D[ind2]
    k2 = w2['kind']
    k = sorted([k1,k2])
    k = k[0]+'-'+k[1]
    s1 = abs(w['orig']-w2['orig'])
    _s1 = abs(w['corrected']-w2['corrected'])
    w['before'] = s1
    w['after'] = _s1
    try:
        cases[k].append(w)
    except:
        cases[k] = [w]

# All possible case pairs (percentage of connections)

In [45]:
s = 0
for w in cases:
    print(w)
    t = len(cases[w])
    s= s + t
    print(t*100.0/32220)
    print('-----')

null-null
76.53631284916202
-----
goodLocal-goodLocal
5.9031657355679705
-----
0distance-0distance
3.258845437616387
-----
badLocal-goodLocal
3.047796399751707
-----
goodLocal-null
7.492240844196152
-----
badLocal-null
3.339540657976412
-----
badLocal-badLocal
0.42209807572936064
-----


# Asymmetry graphs before and after the correction

In [50]:
t = 'badLocal-badLocal'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [61]:
t = 'badLocal-goodLocal'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [51]:
t = 'goodLocal-goodLocal'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [52]:
t = 'goodLocal-null'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [53]:
t = 'badLocal-null'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [55]:
t = 'null-null'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in '+t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)

In [60]:
t = '0distance-0distance'
w = cases[t]
mp = list(map(lambda x:x['before'],w))
p = figure(plot_width=400, plot_height=400,title = 'Asymmetry in '+ t + ' :No Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = mp
hist, edges = np.histogram(measured, density=True, bins=50)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:x['after'],w))
p1 = figure(plot_width=400, plot_height=400,title = 'Asymmetry in '+ t + ' :with Correction',y_axis_label = "Distribution",x_axis_label = "Difference(abs)")
q1,q2 = np.percentile(mp,[0,100])
measured = mp
hist, edges = np.histogram(measured, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p, p1]])
show (grid)